# FB Graphic API 粉絲團統計數據爬蟲
#### 第一部份：取粉絲團基本資料
#### 第二部份：取貼文的統計
#### 先安裝pip3 install requests

In [3]:
import requests
import json
import pandas as pd

## 第一部份：抓取粉絲團基本資料

In [148]:
# 貼上FB API Token
# 簡單測試token是否有通
token = 'EAACEdEose0cBAKs50ZAOFSBRo6bRy7aS2fviPsxwwAjPXyWZCGfjQHhyNNzhtJOZB7i5prHRufOyMzH0aY3P72UNZBi7czZBkyKClV22pQb2Cls6wtKudqSBYxek01EOez6mh3AsnZCo8Oyr3ZC8ZCYjgDlwAfCjzVCC8ZAVsZCmkcuMEwfVkJtbuhBWNMQzm40nWSLYZCbG4yIfQZDZD'
res = requests.get("https://graph.facebook.com/v2.11/me?access_token=%s" % (token))
print(res.text)

{"name":"Alicia Lee","id":"844389662238733"}


In [90]:
# 提取基本資料

# 目標粉絲團縮名，在array中給予縮名的清單
pages = ['BaiChiGongZhu']

# 開始提取粉絲頁基本資料
for page in pages:
    
    # API指令
    info = '?fields=id,name,about,category,likes,link,talking_about_count,website'
    # requests.get取值
    res = requests.get('https://graph.facebook.com/%s%s&access_token=%s' % (page, info, token))
    #print(res.text)
    #json.loads(res.text)['id']

    
    pg1 = [page,                         # 縮名，string
      json.loads(res.text)['id'],        # 唯一碼，numeric
      json.loads(res.text)['name'],      # 粉專名稱，string
      json.loads(res.text)['likes'],     # 粉絲數，numeric
      json.loads(res.text)['talking_about_count'],   # 討論數，numeric
      json.loads(res.text)['category'],  # 分類，string
      json.loads(res.text)['about']      # 介紹，string
          ]     
    

In [ ]:
# 轉成DataFrame
df = pd.DataFrame([pg1])

In [94]:
# 為dataframe欄位命名
df = df.rename(columns={0:'PID',1:'ID',2:'NAME',3:'LIKES',4:'TALKA',5:'TYPE',6:'ABOUT'})

In [95]:
# 增加一row到dataframe中
df = df.append([pg1], ignore_index=True)

/anaconda/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [96]:
df

,PID,ID,NAME,LIKES,TALKA,TYPE,ABOUT,0,1,2,3,4,5,6
0,BaiChiGongZhu,363734540324168,白癡公主,1011428.0,26513.0,Producer,舒服合作信箱87gongzhu@gmail.com\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,test01,9187298370419,beautiful,128793.0,1987239.0,test,test test test,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BaiChiGongZhu,363734540324168,白癡公主,1011428.0,26513.0,Producer,舒服合作信箱87gongzhu@gmail.com\n
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test01,9187298370419,beautiful,128793.0,1987239.0,test,test test test


## 第二部份：取貼文的統計

In [189]:
# 先創一個空的DataFrame
df_article = pd.DataFrame()


In [190]:
# 粉絲團縮名
pages = ['BaiChiGongZhu']


# 近兩週貼文基本統計
for page in pages:
    
    # API指令
    info = '?fields=feed{from, id, type, created_time, message, source, shares}'
    # 近兩週，2017/12/15-12/30
    period = '&since=1513296000&until=1514678400'            
    # requests.get取值
    res = requests.get('https://graph.facebook.com/%s%s%s&access_token=%s' % (page, info, period, token))
    
    
    for i in range(0,len(json.loads(res.text)['feed']['data'])):
        js1 = json.loads(res.text)['feed']['data'][i]
        #print(i)

        feed = [page,                                      # 粉專縮名
               js1['from']['name'],       # 粉專名稱
               js1['id'],                 # 文章唯一碼
               js1['type'],               # 文章類型
               js1['created_time'],       # 貼文時間
               js1['message'],            # 貼文內容
               ]

        if js1.get('source') :
            feed.append(js1['source'])    # 貼文內容的外部來源
        else:
            feed.append(int("0"))
            
        if js1.get('shares') :
            feed.append(js1['shares']['count'])  # shares
        else:
            feed.append(int("0"))       

            
            
        sid = js1['id']
        likes = '?fields=likes.limit(0).summary(total_count)'
        res2 = requests.get('https://graph.facebook.com/%s%s&access_token=%s' % (sid, likes, token))
        react = '?fields=reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry)'
        res3 = requests.get('https://graph.facebook.com/v2.11/%s%s&access_token=%s' % (sid, react, token))
        
        js2 = json.loads(res2.text)
        js3 = json.loads(res3.text)

        feed.append(js2['likes']['summary']['total_count'])      # 讚
        feed.append(js3['reactions_love']['summary']['total_count'])    # reaction_love
        feed.append(js3['reactions_haha']['summary']['total_count'])     # reaction_haha
        feed.append(js3['reactions_sad']['summary']['total_count'])      # reaction_sad
        feed.append(js3['reactions_wow']['summary']['total_count'])      # reaction_wow
        feed.append(js3['reactions_angry']['summary']['total_count'])    # reaction_angry
        
        # 增加一row到dataframe中
        df_article = df_article.append([feed], ignore_index=True)
    

In [191]:
# 為文章dataframe命名  
df_article = df_article.rename(columns={0:'PID',1:'NAME',2:'SID',3:'TYPE',4:'CREATE_TIME',5:'MESSAGE',6:'SOURCE',7:'SHARES',8:'LIKES',9:'LOVE',10:'HAHA',11:'SAD',12:'WOW',13:'ANGRY'})
# show
df_article

,PID,NAME,SID,TYPE,CREATE_TIME,MESSAGE,SOURCE,SHARES,LIKES,LOVE,HAHA,SAD,WOW,ANGRY
0,BaiChiGongZhu,白癡公主,363734540324168_1775153095848965,video,2018-01-01T15:32:45+0000,2017痴痴的人生跑馬燈完整版：https://picsee.pro/v-2896031\n...,https://www.youtube.com/embed/5z3hvyNljd4?list...,10,2455,56,112,0,2,0
1,BaiChiGongZhu,白癡公主,363734540324168_1774334435930831,video,2017-12-31T23:10:16+0000,2017痴痴的人生跑馬燈完整版：https://picsee.pro/v-2896031\n...,https://video.xx.fbcdn.net/v/t42.1790-2/262079...,40,8604,182,631,2,4,0
2,BaiChiGongZhu,白癡公主,363734540324168_1773913249306283,photo,2017-12-31T16:04:40+0000,2018第一PO！\n#美顏開最強,0,4,9531,226,689,0,11,1
3,BaiChiGongZhu,白癡公主,363734540324168_1773139079383700,video,2017-12-30T23:17:55+0000,#菜鳥職劇場 EP13：別人有的是背景，我卻只能看背影完整版：https://picsee....,https://www.youtube.com/embed/W6KZFEY1Asw?list...,8,2403,27,110,0,3,0
4,BaiChiGongZhu,白癡公主,363734540324168_1772638696100405,video,2017-12-30T12:20:40+0000,🍼澳門雙皮燉奶給你呼呼完整版：https://picsee.pro/v-2883558\n\...,https://video.xx.fbcdn.net/v/t42.1790-2/261263...,30,6109,119,223,0,7,0
5,BaiChiGongZhu,白癡公主,363734540324168_1770718982959043,video,2017-12-28T13:33:29+0000,好鬧辣wwww\n這真的可以嗎？只敢看前3秒…\n\n#記者大大是阿甜粉,https://scontent.xx.fbcdn.net/v/t42.1790-4/261...,1,3488,70,506,1,4,0
6,BaiChiGongZhu,白癡公主,363734540324168_1770362939661314,link,2017-12-28T03:18:59+0000,希望不同的內容你們都支持me繼續創作內～\n#阿甜 #架勢十足\n\nhttps://sta...,0,22,8931,235,175,0,9,0
7,BaiChiGongZhu,白癡公主,363734540324168_1765439383487003,video,2017-12-23T01:34:53+0000,走出戶外的87萬訂閱Q&A完整版：https://picsee.pro/v-2780545\...,https://www.youtube.com/embed/hVpmnkJxA5U?list...,9,3150,125,65,1,1,0
8,BaiChiGongZhu,白癡公主,363734540324168_1763402230357385,video,2017-12-20T23:05:45+0000,87萬訂閱QA完整版：https://picsee.pro/v-2780545\n公開me的...,https://video.xx.fbcdn.net/v/t42.1790-2/255641...,42,8520,257,195,0,8,1
9,BaiChiGongZhu,白癡公主,363734540324168_1762071380490470,video,2017-12-19T14:30:26+0000,暴走企鵝興奮的模樣很阿甜😆,https://video.xx.fbcdn.net/v/t42.1790-2/255716...,27,9363,205,2782,1,5,1


# 以下是FB API fanpage相關參數
### 參考用，非正式語法

In [32]:
# 測試ID 
page = 'BaiChiGongZhu'
#page = 'teemomustdie'
#page = '168161663222590'

# 只算LIKE數
#commend = '/feed?fields=message,likes.limit(1).summary(true)'
# 某段時間內的貼文
#commend = '/feed?fields=message,likes.limit(1),created_time&until=1514462400&since=1513771200&limit=100'
# 粉絲團基本資料
commend = '?fields=id,name,about,category,likes,link,talking_about_count,website'
df = df.rename(columns={0:'PID',1:'ID',2:'NAME',3:'LIKES',4:'TALKA',5:'TYPE',6:'ABOUT'})
res = requests.get("https://graph.facebook.com/%s%s&access_token=%s" %(page, commend, token))
print(res.text)

{"id":"363734540324168","name":"\u767d\u7661\u516c\u4e3b","about":"\u8212\u670d\u5408\u4f5c\u4fe1\u7bb187gongzhu\u0040gmail.com\n","category":"Producer","likes":1010895,"link":"https:\/\/www.facebook.com\/BaiChiGongZhu\/","talking_about_count":18458,"website":"http:\/\/pics.ee\/87yt"}


In [ ]:
# 粉絲數
commend = '?fields=fan_count'
# 粉絲頁縮名
commend = '?fields=page_token'


In [ ]:
# 近十篇貼文
commend = 'BaiChiGongZhu?fields=feed.limit(10)'
# 貼文分享自其他來源 {parenet_id}
commend = 'BaiChiGongZhu?fields=feed.limit(10){parent_id}'

# 貼文的ID {id}
# 貼文中tag其他人 {message_tags}
# 貼文中影片的名稱 {name}
# 貼文類型 {icon}  | /icon/post.gif 一般貼文(可能附有照片或影片連結或新聞連結) | /icon/video.gif 影片貼文 | NA 分享貼文
# 貼文類型 {status_type} | added_photos | added_video | mobile_status_update | shared_story |
# 貼文類型 {type} | photo | video | link |
# 原貼文的內容文字 {description} 如果是分享自其他來源就會是原貼文的文字
# 粉絲團貼文內容 {message} 分享的貼文，附加的文字說明在此
# 貼文時間 {created_time}
# 貼文唯一碼 {created_time}
# 哪個粉絲團的貼文 {from} 查單一粉絲團時用不到
# 貼文的預設靜態圖 {full_picture}
# 貼文的導向連結 {link} 可以是內部或外部
# 貼文內容的外部來源 {source}
# 分享次數 {shares}
# 最後加上時間區間 &since=1480849200&until=1480856400  使用timestamp

In [ ]:
# 某貼文的互動數，所有reaction
commend = '363734540324168_1763402230357385?fields=reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry)'
# 文章觀看數 ?fields=seen 沒有權限
# 文章分析  ?fields=insights 沒有權限
